In [47]:
%load_ext autoreload
%autoreload 2
from helper_fns import *
# TEST FUNCTIONS
# Test instap
RT_col = routh_table([1, 2, 3, 4, 5])
print('RT first column')
print(RT_col)

# Test lyap
A = np.array([[-0.1, 0, 1],[0, 1, 0], [0, 0, 1]])
Q = np.array([[1, 0, 0],[0, 1, 0], [0, 3, 1]])
X, _ = lyap(A,Q)
print('LYAP')
print(X)

# Test H2
A = np.array([[-0.5, 1],[-0.2, -0.1]])
B = np.array([[0],[.3]])
C = np.array([[3, 0]])
h2_res, _ = h2(A, B, C)
print('H2')
print(h2_res)

# Test tf2ss
#num = [1, 3]
#den = [1, 5, 10, 20, 20]
#A, B, C = tf2ss(num, den)
#print(h2(A, B, C))

# Test symbolic diff
b = ca.MX.sym('b')
k = ca.MX.sym('k')
A = ca.MX.zeros(2,2)
A[0,1] = ca.MX(1.0)
A[1,0] = -b
A[1,1] = -k
#Q = ca.MX.eye(2)
norm, sol = h2(A, B, C, 'lapackqr')
norm_fn = ca.Function('norm', [b, k], [norm])
b_grad_fn = ca.Function('b_grad',[b, k], [ca.gradient(norm,b)])
k_grad_fn = ca.Function('k_grad',[b, k], [ca.gradient(norm,k)])

b0 = 2
k0 = 1.0
h = 0.01
h20 = norm_fn(b0, k0)
bgrad = b_grad_fn(b0, k0)
kgrad = k_grad_fn(b0, k0)
h21 = norm_fn(b0, k0+h*bgrad+h*kgrad)


print('H2 init: {}'.format(h20))
print('Grads:   {}, {}'.format(bgrad, kgrad))
print('H2 fin:  {}'.format(h21))
print('err:     {}'.format(h20+h*bgrad+h*kgrad-h21))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
RT first column
[1, 2, 1.0, -6.0, 5.0]
LYAP

[[10.5556, 1.66667, 0.555556], 
 [0, -0.5, 0], 
 [0.555556, -1.5, -0.5]]
H2
1.64317
H2 init: 0.45
Grads:   -0.1125, -0.225
H2 fin:  0.450761
err:     -0.0041363


In [46]:
# Test the routh-table based stability bound
# comparing aginst http://control.asu.edu/Classes/MAE318/318Lecture10.pdf
%load_ext autoreload
%autoreload 2
RT_col = routh_table([1, ca.MX.sym('a'), ca.MX.sym('b'), ca.MX.sym('c')])
print(RT_col)
RT_col = routh_table([1, 10, 31, 30+ca.MX.sym('k')])
print(RT_col)
RT_col = routh_table([1, 2, 2, 4, 11, 10]) # shouuld cauuse zero
print(RT_col)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(0==(((a*b)-c)/a))
@1=(((a*b)-c)/a), (0==((@1*c)/@1))
[1, MX(a), MX((((a*b)-c)/a)), MX(@1=(((a*b)-c)/a), ((@1*c)/@1))]
(0==((310-(30+k))/10))
@1=(30+k), @2=((310-@1)/10), (0==((@2*@1)/@2))
[1, 10, MX(((310-(30+k))/10)), MX(@1=(30+k), @2=((310-@1)/10), ((@2*@1)/@2))]
True
Routh Table has leading 0; interrupting, may not be complete
[1, 2]


In [9]:
# Testing the lyap_solver, comparing with the vanilla Casadi Solve
from autodiff_sys import Sys
from helper_fns import lyap
 
A = np.array([[0.1, 1],[1, 3]])
Q = np.eye(2)
B = np.array([[1.3, 0.1],[0.1, 5]])
X, _ = lyap(A, B, 'scipy')
print('Scipy lyap:')
print(X)
X, _ = lyap(A, B, 'lapackqr')
print('Casadi lyap:')
print(X)


A = np.array([[-0.5, 1],[-0.2, -0.1]])
#B = np.array([[0],[.3]])
#C = np.array([[3, 0]])
n, _ = h2(A, B, C, 'scipy')
print('Scipy h2')
print(n)
n, _ = h2(A, B, C, 'lapackqr')
print('Casadi h2')
print(n)


As = ca.MX.sym('As',2,2)
Qs = ca.MX.sym('Qs',2,2)
A2s = ca.MX.sym('As',2,2)
Q2s = ca.MX.sym('Qs',2,2)
Ps = ca.MX.sym('Ps',2,2)
fseeds = [ca.MX.sym("f",2,2) for i in range(2)]
aseeds = [ca.MX.sym("a",2,2)  for i in range(1)]
#res = ly.call([As, Qs], True)
#res = ly(As, Qs)
res, sol = lyap(As, Qs, 'scipy')
res_ca, _ = lyap(As, Qs, 'lapackqr')

[rev] = ca.reverse([res], [As, Qs, Ps, A2s, Q2s], [aseeds], {})
[rev_ca] = ca.reverse([res_ca], [As, Qs, Ps, A2s, Q2s], [aseeds], {})

[fwd] = ca.forward([res], [As, Qs], [fseeds], {})
[fwd_ca] = ca.forward([res_ca], [As, Qs], [fseeds], {})

fwd_fn = ca.Function('fwfn', [As, Qs, *fseeds], fwd)
fwd_ca_fn = ca.Function('fwfn', [As, Qs, *fseeds], fwd_ca)
rev_fn = ca.Function('rvfn', [As, Qs, *aseeds], rev)
rev_ca_fn = ca.Function('rvfn', [As, Qs, *aseeds], rev_ca)
print('Validating lyap fwd')
print('Scipy')
print(fwd_fn(A, Q, A, B))
print('Casadi')
print(fwd_ca_fn(A, Q, A, B))
print('Validating lyap rev')
print('Scipy')
print(rev_fn(A, Q, B))
print('Casadi')
print(rev_ca_fn(A, Q, B))

Scipy lyap:

[[3.5, -1], 
 [-1, -0.5]]
Casadi lyap:

[[3.5, -1], 
 [-1, -0.5]]
Scipy h2
27.7009
Casadi h2
27.7009
Validating lyap fwd
Scipy

[[13.66, 6.68], 
 [6.68, 6.64]]
Casadi

[[13.66, 6.68], 
 [6.68, 6.64]]
Validating lyap rev
Scipy
(DM(
[[11.112, 1.376], 
 [31.52, 42.96]]), DM(
[[1.78, -1.2], 
 [-1.2, 13]]), DM(
[[00, 00], 
 [00, 00]]), DM(
[[00, 00], 
 [00, 00]]), DM(
[[00, 00], 
 [00, 00]]))
Casadi
(DM(
[[11.112, 1.376], 
 [31.52, 42.96]]), DM(
[[1.78, -1.2], 
 [-1.2, 13]]), DM(
[[00, 00], 
 [00, 00]]), DM(
[[00, 00], 
 [00, 00]]), DM(
[[00, 00], 
 [00, 00]]))


In [ ]:
# Find extrema of imaginary component over frequency
imag_der = der(imag_coeff)
print(imag_coeff)
poly_coeffs = ca.SX.sym('poly',3,1)
poly_coeffs[0] = imag_der[0]
poly_coeffs[1] = imag_der[2]
poly_coeffs[2] = imag_der[4]
roots = ca.sqrt(-ca.poly_roots(poly_coeffs))
print(roots[0])

for j in range(roots.shape[0]):
    tot = 0.0
    num_coeffs = len(imag_coeff)
    for i in range(num_coeffs):
        tot += ca.constpow(roots[j],num_coeffs-i-1)*imag_coeff[i]
    print(tot)
crit_points_imag = []
crit_points_real = []
crit_points_freq = []
for i in range(roots.shape[0]):
    crit_points_freq.append(ca.Function('crit_freq', [M, B, Kp, Kd, Ma, Ba, Kl], [roots[i]]))
    crit_points_imag.append(ca.Function('crit_imag', [M, B, Kp, Kd, Ma, Ba, Kl], [imag_fn(roots[i], M, B, Kp, Kd, Ma, Ba, Kl)]))
    crit_points_real.append(ca.Function('crit_real', [M, B, Kp, Kd, Ma, Ba, Kl], [real_fn(roots[i], M, B, Kp, Kd, Ma, Ba, Kl)]))

# Verifying the real/imaginary components are realistic
# Can compare with addmittance_check_casadi.m
import numpy as np
import matplotlib.pyplot as plt
Mt = 5
Bt = 10
Kpt = 1
Kdt = 2
Mat = 3
Bat = 5
Klt = 0.1

plt.figure()
plt.clf()
for om in [0.001, 10, 500]:
    print("freq {} real {} im {}".format(om, real_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), imag_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt)))

for om in np.logspace(-10,10, num = 1000):
    plt.plot(real_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), imag_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt),'ko')
    
for cp_r, cp_i, cp_f in zip(crit_points_real, crit_points_imag, crit_points_freq):
    print('crit point re: {} im: {} fr: {}'.format(\
                        cp_r(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), 
                        cp_i(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt),
                        cp_f(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt)))
    plt.plot(cp_r(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), cp_i(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt),'dr')
plt.show()